<a href="https://colab.research.google.com/github/jcohut/TensorFlow_Practice/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [11]:
# Mount Google Drive directory to access mnist.npz from tmp2 in Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab happy-or-sad.zip from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location

# /content/drive/My Drive/Coursera/TensorFlow/C1_W4_files/happy-or-sad.zip
# Path for Coursera Jupyter autograder:
# path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

# Path to happy-or-sad.zip in Google Drive:
path = f"{getcwd()}/drive/My Drive/Coursera/TensorFlow/C1_W4_files/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [17]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        # Change 'accuracy' to 'acc' for the Coursera autograder!
        if(logs.get('accuracy')>DESIRED_ACCURACY): 
          print("\nReached 99.9% accuracy so cancelling training!")
          self.model.stop_training = True

    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 150x150 with 3 bytes color
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
      
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.001),
                  metrics=['accuracy'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(
          train_generator,
          steps_per_epoch=8,
          epochs=15,
          verbose=1,
          callbacks=[callbacks])
    # model fitting
    # Change 'accuracy' to 'acc' for the Coursera autograder!
    return history.history['accuracy'][-1]

In [18]:
# The Expected output: "Reached 99.9% accuracy so cancelling training!""
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 2s 242ms/step - loss: 1.2830 - accuracy: 0.5250
Epoch 2/15
8/8 [==============================] - 2s 239ms/step - loss: 0.5626 - accuracy: 0.7750
Epoch 3/15
8/8 [==============================] - 2s 240ms/step - loss: 0.2775 - accuracy: 0.9000
Epoch 4/15
8/8 [==============================] - 2s 245ms/step - loss: 0.1342 - accuracy: 0.9500
Epoch 5/15
8/8 [==============================] - 2s 240ms/step - loss: 0.2098 - accuracy: 0.9000
Epoch 6/15
8/8 [==============================] - ETA: 0s - loss: 0.0487 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 240ms/step - loss: 0.0487 - accuracy: 1.0000


1.0